In [1]:
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import os, sys, pickle, glob, gc
from collections import Counter
import itertools
import cudf
print('We will use RAPIDS version',cudf.__version__)
cudf.set_option("default_integer_bitwidth", 32)
cudf.set_option("default_float_bitwidth", 32)

from coVisitation import CV_carts_orders, CV_B2B, CV_clicks
from eventsSuggester import ClicksSuggester, BuysSuggester

We will use RAPIDS version 22.10.01+2.gca9a422da9


In [2]:
import xgboost as xgb
from sklearn.model_selection import GroupKFold

In [3]:
inputPath = '../input/newSplited/'
outputPath = '../output/newSplited/'
! mkdir {outputPath}

mkdir: cannot create directory ‘../output/newSplited/’: File exists


In [4]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}

# Train

## Create covisitation matrix
(Train + Val_A)

In [4]:
files = glob.glob(inputPath + '/parquets/*_parquets/*')
saveFolder = outputPath + '/coVisitationPqt/'
! mkdir {saveFolder}

mkdir: cannot create directory ‘../output/newSplited//coVisitationPqt/’: File exists


In [5]:
data_cache = {}
for f in files: 
    data_cache[f] = pd.read_parquet(f)
data_cache

{'../input/newSplited//parquets/train_parquets/7600000_7700000.parquet':          session      aid          ts  type
 0        7600000    45494  1660352198     0
 1        7600000   214334  1660352290     0
 2        7600000   269405  1660352332     0
 3        7600000   269405  1660352380     0
 4        7600000   269405  1660469933     0
 ...          ...      ...         ...   ...
 1008231  7699999  1388339  1661005635     0
 1008232  7699999  1002712  1661005719     0
 1008233  7699999   381787  1661016030     0
 1008234  7699999   198903  1661099585     0
 1008235  7699999   603386  1661099590     0
 
 [1008236 rows x 4 columns],
 '../input/newSplited//parquets/train_parquets/800000_900000.parquet':          session      aid          ts  type
 0         800000  1067553  1659369726     0
 1         800000  1313522  1659371888     0
 2         800000   964510  1660152084     0
 3         800000   620884  1660152091     0
 4         800000   888891  1660152388     0
 ...          ...

In [6]:
cv_carts_orders = CV_carts_orders(files, data_cache, saveFolder)
cv_carts_orders.processDisks(50)

We will process 112 files, in groups of 3 and chunks of 25.

### DISK PART 1
Processing files 0 thru 24 in groups of 3...
0 , 3 , 6 , 9 , 12 , 15 , 18 , 21 , 24 , 
Processing files 25 thru 49 in groups of 3...
25 , 28 , 31 , 34 , 37 , 40 , 43 , 46 , 49 , 
Processing files 50 thru 74 in groups of 3...
50 , 53 , 56 , 59 , 62 , 65 , 68 , 71 , 74 , 
Processing files 75 thru 99 in groups of 3...
75 , 78 , 81 , 84 , 87 , 90 , 93 , 96 , 99 , 
Processing files 100 thru 111 in groups of 3...
100 , 103 , 106 , 109 , 

### DISK PART 2
Processing files 0 thru 24 in groups of 3...
0 , 3 , 6 , 9 , 12 , 15 , 18 , 21 , 24 , 
Processing files 25 thru 49 in groups of 3...
25 , 28 , 31 , 34 , 37 , 40 , 43 , 46 , 49 , 
Processing files 50 thru 74 in groups of 3...
50 , 53 , 56 , 59 , 62 , 65 , 68 , 71 , 74 , 
Processing files 75 thru 99 in groups of 3...
75 , 78 , 81 , 84 , 87 , 90 , 93 , 96 , 99 , 
Processing files 100 thru 111 in groups of 3...
100 , 103 , 106 , 109 , 

### DISK PART 3
Processing files 

In [7]:
cv_B2B = CV_B2B(files, data_cache, saveFolder)
cv_B2B.processDisks(50)

We will process 112 files, in groups of 3 and chunks of 25.

### DISK PART 1
Processing files 0 thru 24 in groups of 3...
0 , 3 , 6 , 9 , 12 , 15 , 18 , 21 , 24 , 
Processing files 25 thru 49 in groups of 3...
25 , 28 , 31 , 34 , 37 , 40 , 43 , 46 , 49 , 
Processing files 50 thru 74 in groups of 3...
50 , 53 , 56 , 59 , 62 , 65 , 68 , 71 , 74 , 
Processing files 75 thru 99 in groups of 3...
75 , 78 , 81 , 84 , 87 , 90 , 93 , 96 , 99 , 
Processing files 100 thru 111 in groups of 3...
100 , 103 , 106 , 109 , 

### DISK PART 2
Processing files 0 thru 24 in groups of 3...
0 , 3 , 6 , 9 , 12 , 15 , 18 , 21 , 24 , 
Processing files 25 thru 49 in groups of 3...
25 , 28 , 31 , 34 , 37 , 40 , 43 , 46 , 49 , 
Processing files 50 thru 74 in groups of 3...
50 , 53 , 56 , 59 , 62 , 65 , 68 , 71 , 74 , 
Processing files 75 thru 99 in groups of 3...
75 , 78 , 81 , 84 , 87 , 90 , 93 , 96 , 99 , 
Processing files 100 thru 111 in groups of 3...
100 , 103 , 106 , 109 , 


In [8]:
cv_clicks = CV_clicks(files, data_cache, saveFolder)
cv_clicks.processDisks(50)

We will process 112 files, in groups of 3 and chunks of 25.

### DISK PART 1
Processing files 0 thru 24 in groups of 3...
0 , 3 , 6 , 9 , 12 , 15 , 18 , 21 , 24 , 
Processing files 25 thru 49 in groups of 3...
25 , 28 , 31 , 34 , 37 , 40 , 43 , 46 , 49 , 
Processing files 50 thru 74 in groups of 3...
50 , 53 , 56 , 59 , 62 , 65 , 68 , 71 , 74 , 
Processing files 75 thru 99 in groups of 3...
75 , 78 , 81 , 84 , 87 , 90 , 93 , 96 , 99 , 
Processing files 100 thru 111 in groups of 3...
100 , 103 , 106 , 109 , 

### DISK PART 2
Processing files 0 thru 24 in groups of 3...
0 , 3 , 6 , 9 , 12 , 15 , 18 , 21 , 24 , 
Processing files 25 thru 49 in groups of 3...
25 , 28 , 31 , 34 , 37 , 40 , 43 , 46 , 49 , 
Processing files 50 thru 74 in groups of 3...
50 , 53 , 56 , 59 , 62 , 65 , 68 , 71 , 74 , 
Processing files 75 thru 99 in groups of 3...
75 , 78 , 81 , 84 , 87 , 90 , 93 , 96 , 99 , 
Processing files 100 thru 111 in groups of 3...
100 , 103 , 106 , 109 , 

### DISK PART 3
Processing files 

In [9]:
# FREE MEMORY
del data_cache
gc.collect()

0

## Generate 50 candidates for validation data with covisitation

In [10]:
def pqt_to_dict(df):
    return df.groupby('aid_x').aid_y.apply(list).to_dict()

In [11]:
top_50_clicks = {}
for k in range(0, cv_clicks.DISK_PIECES):
    top_50_clicks.update( pqt_to_dict( pd.read_parquet(cv_clicks.saveFolder + f'/top_50_clicks_v{cv_clicks.VER}_{k}.pqt') ) )

top_50_buy2buy = {}
for k in range(0, cv_B2B.DISK_PIECES):
    top_50_buy2buy.update( pqt_to_dict( pd.read_parquet(cv_B2B.saveFolder + f'/top_50_buy2buy_v{cv_B2B.VER}_{k}.pqt') ) )

top_50_buys = {}
for k in range(0, cv_carts_orders.DISK_PIECES): 
    top_50_buys.update( pqt_to_dict( pd.read_parquet(cv_carts_orders.saveFolder + f'/top_50_carts_orders_v{cv_carts_orders.VER}_{k}.pqt') ) )

In [12]:
val_A = pd.read_parquet(inputPath + 'val_A.parquet')
val_A

,session,aid,ts,type
0,8643220,573273,1660514400,0
1,8643221,921137,1660514400,0
2,8643222,930597,1660514400,0
3,8643222,1037630,1660514457,0
4,8643222,756920,1660514485,0
...,...,...,...,...
9710473,11098472,613059,1661119170,0
9710474,11098478,261405,1661119174,0
9710475,11098483,38826,1661119179,0
9710476,11098494,1733630,1661119183,0


In [13]:
top_clicks = val_A.loc[val_A['type']==0,'aid'].value_counts().index.values[:50]
top_orders = val_A.loc[val_A['type']==2,'aid'].value_counts().index.values[:50]

print('Here are size of our 3 co-visitation matrices:')
print( len( top_50_clicks ), len( top_50_buy2buy ), len( top_50_buys ) )

Here are size of our 3 co-visitation matrices:
1774647 1004080 1774647


In [14]:
clicksSuggester = ClicksSuggester(top_clicks, top_50_clicks)
buysSuggester = BuysSuggester(top_orders, top_50_buy2buy, top_50_buys)

In [15]:
%%time

pred_df_clicks = val_A.sort_values(["session", "ts"]).groupby(["session"]).apply(
    lambda x: clicksSuggester.suggest(x, 50)
)

pred_df_buys = val_A.sort_values(["session", "ts"]).groupby(["session"]).apply(
    lambda x: buysSuggester.suggest(x, 50)
)

CPU times: user 13min 35s, sys: 1.98 s, total: 13min 37s
Wall time: 13min 37s


In [16]:
clicks_pred_df = pd.DataFrame(pred_df_clicks, columns=["labels"]).reset_index()
clicks_pred_df

,session,labels
0,8643220,"[573273, 399315, 1308823, 1768884, 1337750, 93..."
1,8643221,"[921137, 1797158, 1543291, 1190772, 661204, 10..."
2,8643222,"[1410436, 756920, 1029653, 1037630, 930597, 14..."
3,8643223,"[1811963, 206418, 986164, 390163, 423037, 6344..."
4,8643224,"[1063371, 154901, 778561, 1106262, 1706288, 15..."
...,...,...
2237920,11098472,"[613059, 980990, 1596292, 1839414, 317654, 110..."
2237921,11098478,"[261405, 25572, 246801, 274542, 466846, 495972..."
2237922,11098483,"[38826, 46115, 1112345, 1452837, 636232, 99381..."
2237923,11098494,"[1733630, 1601525, 155279, 491384, 1562015, 47..."


In [17]:
buys_pred_df = pd.DataFrame(pred_df_buys, columns=["labels"]).reset_index()
buys_pred_df

,session,labels
0,8643220,"[573273, 399315, 1308823, 1768884, 1337750, 93..."
1,8643221,"[921137, 653222, 26362, 216169, 274783, 472920..."
2,8643222,"[1410436, 756920, 1029653, 1037630, 930597, 14..."
3,8643223,"[1811963, 206418, 986164, 390163, 634452, 4230..."
4,8643224,"[1063371, 154901, 778561, 1106262, 1706288, 15..."
...,...,...
2237920,11098472,"[613059, 980990, 1596292, 1839414, 317654, 110..."
2237921,11098478,"[261405, 254434, 274542, 685114, 25572, 76879,..."
2237922,11098483,"[38826, 46115, 1112345, 1452837, 636232, 99381..."
2237923,11098494,"[1733630, 1601525, 491384, 1562015, 155279, 16..."


In [18]:
candidatesSavePath = outputPath + 'top50Candidates/'
! mkdir {candidatesSavePath}
clicks_pred_df.to_parquet(candidatesSavePath + 'clicksCandidate.pqt')
buys_pred_df.to_parquet(candidatesSavePath + 'buysCandidate.pqt')

mkdir: cannot create directory ‘../output/newSplited/top50Candidates/’: File exists


## Create features for validation data

In [4]:
candidatesSavePath = outputPath + 'top50Candidates/'
clicks_pred_df = pd.read_parquet(candidatesSavePath + 'clicksCandidate.pqt')
buys_pred_df = pd.read_parquet(candidatesSavePath + 'buysCandidate.pqt')

clicks_pred_df['session'] = clicks_pred_df.session.astype('int32')
clicks_pred_df['labels'] = clicks_pred_df.labels.apply(lambda x: x.astype(np.int32))
buys_pred_df['session'] = buys_pred_df.session.astype('int32')
buys_pred_df['labels'] = buys_pred_df.labels.apply(lambda x: x.astype(np.int32))

In [5]:
aids = clicks_pred_df.labels.explode().astype('int32').rename('aid')
clicksCandidate = clicks_pred_df[['session']]
clicksCandidate = clicksCandidate.merge(aids, left_index=True, right_index=True, how='left').reset_index(drop=True)
clicksCandidate

,session,aid
0,8643220,573273
1,8643220,399315
2,8643220,1308823
3,8643220,1768884
4,8643220,1337750
...,...,...
111896245,11098509,1530242
111896246,11098509,1661756
111896247,11098509,933444
111896248,11098509,1163543


In [6]:
aids = buys_pred_df.labels.explode().astype('int32').rename('aid')
buysCandidate = buys_pred_df[['session']]
buysCandidate = buysCandidate.merge(aids, left_index=True, right_index=True, how='left').reset_index(drop=True)
buysCandidate

,session,aid
0,8643220,573273
1,8643220,399315
2,8643220,1308823
3,8643220,1768884
4,8643220,1337750
...,...,...
111896245,11098509,970614
111896246,11098509,1455953
111896247,11098509,68215
111896248,11098509,508191


In [7]:
del clicks_pred_df, buys_pred_df, aids
gc.collect()

0

In [21]:
train_df = pd.read_parquet(inputPath + 'train.parquet')
train_df

,session,aid,ts,type
0,0,1517085,1659304800,0
1,0,1563459,1659304904,0
2,0,1309446,1659367439,0
3,0,16246,1659367719,0
4,0,1781822,1659367871,0
...,...,...,...,...
144332289,8643218,12236,1660514471,0
144332290,8643218,1072581,1660514631,0
144332291,8643218,668368,1660514674,0
144332292,8643219,1112446,1660514399,0


In [22]:
val_A = pd.read_parquet(inputPath + 'val_A.parquet')
val_A

,session,aid,ts,type
0,8643220,573273,1660514400,0
1,8643221,921137,1660514400,0
2,8643222,930597,1660514400,0
3,8643222,1037630,1660514457,0
4,8643222,756920,1660514485,0
...,...,...,...,...
9710473,11098472,613059,1661119170,0
9710474,11098478,261405,1661119174,0
9710475,11098483,38826,1661119179,0
9710476,11098494,1733630,1661119183,0


In [23]:
train_val_df = pd.concat([train_df, val_A], axis=0, ignore_index=True)

In [24]:
featuresPath = outputPath + 'features/'
! mkdir {featuresPath}

mkdir: cannot create directory ‘../output/newSplited/features/’: File exists


### Item features

In [25]:
item_features = train_val_df.groupby('aid').agg({'aid':'count', 'session':'nunique', 'type':'mean'})
item_features.columns = ['item_item_count', 'item_user_count', 'item_buy_ratio']
item_features

,item_item_count,item_user_count,item_buy_ratio
aid,,,
0,30,24,0.000000
1,32,29,0.031250
2,10,10,0.000000
3,893,524,0.071669
4,133,92,0.037594
...,...,...,...
1855598,6,6,0.000000
1855599,8,8,0.000000
1855600,63,38,0.079365


In [26]:
item_features.to_parquet(featuresPath + 'item_features.pqt')

### User features

In [27]:
user_features = val_A.groupby('session').agg({'session':'count', 'aid':'nunique', 'type':'mean'})
user_features.columns = ['user_user_count', 'user_item_count', 'user_buy_ratio']
user_features

,user_user_count,user_item_count,user_buy_ratio
session,,,
8643220,1,1,0.000000
8643221,1,1,0.000000
8643222,7,5,0.000000
8643223,3,2,0.000000
8643224,31,13,0.354839
...,...,...,...
11098472,1,1,0.000000
11098478,1,1,0.000000
11098483,1,1,0.000000


In [28]:
user_features.to_parquet(featuresPath + 'user_features.pqt')

### Merge features

In [8]:
featuresPath = outputPath + 'features/'
item_features = pd.read_parquet(featuresPath + 'item_features.pqt')
user_features = pd.read_parquet(featuresPath + 'user_features.pqt')

item_features.item_item_count = item_features.item_item_count.astype('int32')
item_features.item_user_count = item_features.item_user_count.astype('int32')
user_features.user_user_count = user_features.user_user_count.astype('int16')
user_features.user_item_count = user_features.user_item_count.astype('int16')

In [9]:
clicksCandidate = clicksCandidate.merge(item_features, left_on='aid', right_index=True, how='left').fillna(-1)
clicksCandidate = clicksCandidate.merge(user_features, left_on='session', right_index=True, how='left').fillna(-1)
clicksCandidate

,session,aid,item_item_count,item_user_count,item_buy_ratio,user_user_count,user_item_count,user_buy_ratio
0,8643220,573273,10228,5146,0.154967,1,1,0.0
1,8643220,399315,35662,20110,0.108042,1,1,0.0
2,8643220,1308823,9083,4740,0.167015,1,1,0.0
3,8643220,1768884,10765,5134,0.111194,1,1,0.0
4,8643220,1337750,5474,3316,0.053708,1,1,0.0
...,...,...,...,...,...,...,...,...
111896245,11098509,1530242,2488,1419,0.114148,1,1,0.0
111896246,11098509,1661756,490,327,0.030612,1,1,0.0
111896247,11098509,933444,7901,3439,0.155803,1,1,0.0
111896248,11098509,1163543,806,471,0.094293,1,1,0.0


In [10]:
buysCandidate = buysCandidate.merge(item_features, left_on='aid', right_index=True, how='left').fillna(-1)
buysCandidate = buysCandidate.merge(user_features, left_on='session', right_index=True, how='left').fillna(-1)
buysCandidate

,session,aid,item_item_count,item_user_count,item_buy_ratio,user_user_count,user_item_count,user_buy_ratio
0,8643220,573273,10228,5146,0.154967,1,1,0.0
1,8643220,399315,35662,20110,0.108042,1,1,0.0
2,8643220,1308823,9083,4740,0.167015,1,1,0.0
3,8643220,1768884,10765,5134,0.111194,1,1,0.0
4,8643220,1337750,5474,3316,0.053708,1,1,0.0
...,...,...,...,...,...,...,...,...
111896245,11098509,970614,274,201,0.036496,1,1,0.0
111896246,11098509,1455953,587,320,0.156729,1,1,0.0
111896247,11098509,68215,242,155,0.107438,1,1,0.0
111896248,11098509,508191,41,31,0.000000,1,1,0.0


### add label

In [11]:
val_labels_pd = pd.read_parquet(inputPath + 'val_B.parquet')
val_labels_pd

,session,type,ground_truth
0,8643220,clicks,[1189975]
1,8643221,clicks,[111691]
2,8643222,clicks,"[1104129, 1568399, 1772730, 1772730, 1104129]"
3,8643223,clicks,[1742315]
4,8643224,clicks,[471589]
...,...,...,...
2756163,11098472,clicks,[504876]
2756164,11098478,clicks,[261405]
2756165,11098483,clicks,[46115]
2756166,11098494,clicks,[1733630]


In [12]:
val_labels_clicks = val_labels_pd.loc[val_labels_pd['type'] == 'clicks']
aids = val_labels_clicks.ground_truth.explode().astype('int32').rename('aid')
val_labels_clicks = val_labels_clicks[['session']].astype('int32')
val_labels_clicks = val_labels_clicks.merge(aids, left_index=True, right_index=True, how='left')
val_labels_clicks['click'] = 1
val_labels_clicks

,session,aid,click
0,8643220,1189975,1
1,8643221,111691,1
2,8643222,1104129,1
2,8643222,1568399,1
2,8643222,1772730,1
...,...,...,...
2756163,11098472,504876,1
2756164,11098478,261405,1
2756165,11098483,46115,1
2756166,11098494,1733630,1


In [13]:
val_labels_carts = val_labels_pd.loc[val_labels_pd['type'] == 'carts']
aids = val_labels_carts.ground_truth.explode().astype('int32').rename('aid')
val_labels_carts = val_labels_carts[['session']].astype('int32')
val_labels_carts = val_labels_carts.merge(aids, left_index=True, right_index=True, how='left')
val_labels_carts['cart'] = 1
val_labels_carts

,session,aid,cart
8,8643226,1845885,1
12,8643228,1698483,1
18,8643232,543567,1
18,8643232,1539992,1
18,8643232,1520883,1
...,...,...,...
2756118,11098382,101982,1
2756142,11098423,1082823,1
2756142,11098423,1082823,1
2756154,11098444,1665618,1


In [14]:
val_labels_orders = val_labels_pd.loc[val_labels_pd['type'] == 'orders']
aids = val_labels_orders.ground_truth.explode().astype('int32').rename('aid')
val_labels_orders = val_labels_orders[['session']].astype('int32')
val_labels_orders = val_labels_orders.merge(aids, left_index=True, right_index=True, how='left')
val_labels_orders['order'] = 1
val_labels_orders

,session,aid,order
5,8643224,1502227,1
5,8643224,631899,1
5,8643224,945574,1
9,8643226,1845885,1
14,8643229,739433,1
...,...,...,...
2755982,11098185,230910,1
2756056,11098290,589365,1
2756056,11098290,878689,1
2756076,11098316,1460737,1


In [15]:
clicksCandidate = clicksCandidate.merge(val_labels_clicks, on=['session','aid'],how='left').fillna(0)
clicksCandidate

,session,aid,item_item_count,item_user_count,item_buy_ratio,user_user_count,user_item_count,user_buy_ratio,click
0,8643220,573273,10228,5146,0.154967,1,1,0.0,0.0
1,8643220,399315,35662,20110,0.108042,1,1,0.0,0.0
2,8643220,1308823,9083,4740,0.167015,1,1,0.0,0.0
3,8643220,1768884,10765,5134,0.111194,1,1,0.0,0.0
4,8643220,1337750,5474,3316,0.053708,1,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...
112890380,11098509,1530242,2488,1419,0.114148,1,1,0.0,0.0
112890381,11098509,1661756,490,327,0.030612,1,1,0.0,0.0
112890382,11098509,933444,7901,3439,0.155803,1,1,0.0,0.0
112890383,11098509,1163543,806,471,0.094293,1,1,0.0,0.0


In [18]:
cartsCandidate = buysCandidate.merge(val_labels_carts, on=['session','aid'],how='left').fillna(0)
cartsCandidate

,session,aid,item_item_count,item_user_count,item_buy_ratio,user_user_count,user_item_count,user_buy_ratio,cart
0,8643220,573273,10228,5146,0.154967,1,1,0.0,0.0
1,8643220,399315,35662,20110,0.108042,1,1,0.0,0.0
2,8643220,1308823,9083,4740,0.167015,1,1,0.0,0.0
3,8643220,1768884,10765,5134,0.111194,1,1,0.0,0.0
4,8643220,1337750,5474,3316,0.053708,1,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...
111944086,11098509,970614,274,201,0.036496,1,1,0.0,0.0
111944087,11098509,1455953,587,320,0.156729,1,1,0.0,0.0
111944088,11098509,68215,242,155,0.107438,1,1,0.0,0.0
111944089,11098509,508191,41,31,0.000000,1,1,0.0,0.0


In [20]:
ordersCandidate = buysCandidate.merge(val_labels_orders, on=['session','aid'],how='left').fillna(0)
ordersCandidate

,session,aid,item_item_count,item_user_count,item_buy_ratio,user_user_count,user_item_count,user_buy_ratio,order
0,8643220,573273,10228,5146,0.154967,1,1,0.0,0.0
1,8643220,399315,35662,20110,0.108042,1,1,0.0,0.0
2,8643220,1308823,9083,4740,0.167015,1,1,0.0,0.0
3,8643220,1768884,10765,5134,0.111194,1,1,0.0,0.0
4,8643220,1337750,5474,3316,0.053708,1,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...
111913364,11098509,970614,274,201,0.036496,1,1,0.0,0.0
111913365,11098509,1455953,587,320,0.156729,1,1,0.0,0.0
111913366,11098509,68215,242,155,0.107438,1,1,0.0,0.0
111913367,11098509,508191,41,31,0.000000,1,1,0.0,0.0


In [16]:
data4xgb_path = outputPath + 'data4xgb/'
! mkdir {data4xgb_path}

mkdir: cannot create directory ‘../output/newSplited/data4xgb/’: File exists


In [17]:
clicksCandidate.to_parquet(data4xgb_path + 'data4xgb_clicks.pqt')

In [19]:
cartsCandidate.to_parquet(data4xgb_path + 'data4xgb_carts.pqt')

In [21]:
ordersCandidate.to_parquet(data4xgb_path + 'data4xgb_orders.pqt')

## Train model

In [30]:
data4xgb_path = outputPath + 'data4xgb/'
predictType = 'orders'
data4xgb = pd.read_parquet(data4xgb_path + 'data4xgb_' + predictType + '.pqt')
data4xgb

,session,aid,item_item_count,item_user_count,item_buy_ratio,user_user_count,user_item_count,user_buy_ratio,order
0,8643220,573273,10228,5146,0.154967,1,1,0.0,0.0
1,8643220,399315,35662,20110,0.108042,1,1,0.0,0.0
2,8643220,1308823,9083,4740,0.167015,1,1,0.0,0.0
3,8643220,1768884,10765,5134,0.111194,1,1,0.0,0.0
4,8643220,1337750,5474,3316,0.053708,1,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...
111913364,11098509,970614,274,201,0.036496,1,1,0.0,0.0
111913365,11098509,1455953,587,320,0.156729,1,1,0.0,0.0
111913366,11098509,68215,242,155,0.107438,1,1,0.0,0.0
111913367,11098509,508191,41,31,0.000000,1,1,0.0,0.0


In [31]:
positives = data4xgb.loc[data4xgb.iloc[:, -1] == 1]
print(len(positives))
negatives = data4xgb.loc[data4xgb.iloc[:, -1] == 0].sample(frac=0.05)
print(len(negatives))
data4xgb_underSample = pd.concat([positives,negatives],axis=0,ignore_index=True)

292434
5581047


In [5]:
modelSavedPAth = outputPath + 'savedModel/xgbAllTrain/'
! mkdir -p {modelSavedPAth}

In [6]:
predictType = 'orders'
data4xgb_path = outputPath + 'data4xgb/'
data4xgb = pd.read_parquet(data4xgb_path + 'data4xgb_' + predictType + '.pqt')

positives = data4xgb.loc[data4xgb.iloc[:, -1] == 1]
print(len(positives))
negatives = data4xgb.loc[data4xgb.iloc[:, -1] == 0].sample(frac=0.05)
print(len(negatives))
data4xgb_underSample = pd.concat([positives,negatives],axis=0,ignore_index=True)

data4xgb_val_path = '../output/data4xgb/'
data4xgb_val = pd.read_parquet(data4xgb_val_path + 'data4xgb_' + predictType + '.pqt')
data4xgb_val_underSample = data4xgb_val.sample(frac=0.1)
data4xgb_val_underSample


292434
5581047


,session,aid,item_item_count,item_user_count,item_buy_ratio,user_user_count,user_item_count,user_buy_ratio,order
28785801,11674244,1067424,1442,1134,0.063800,2,2,0.000000,0.0
54508076,12188689,1383529,32384,21438,0.114069,23,15,0.434783,0.0
62182211,12342172,526233,345,276,0.026087,1,1,0.000000,0.0
10829956,11315127,468768,1935,1211,0.074419,3,3,0.000000,0.0
44430766,11987143,1373672,247,187,0.125506,3,2,0.000000,0.0
...,...,...,...,...,...,...,...,...,...
52123989,12141007,97436,155,104,0.032258,1,1,0.000000,0.0
19327993,11485087,1255391,2138,1494,0.151543,1,1,0.000000,0.0
34738517,11793298,881286,32371,18180,0.087949,1,1,0.000000,0.0
55840594,12215339,303312,2827,1343,0.157057,1,1,0.000000,0.0


In [33]:
skf = GroupKFold(n_splits=5)
topN = 50
candidates = data4xgb_underSample
modelName = 'xgb_' + predictType

predsVal = np.zeros(len(candidates))

for fold,(train_idx, valid_idx) in enumerate(skf.split(candidates, candidates.iloc[:, -1], groups=candidates['session'] )):
    print('========= type', predictType, '==========')
    print('========= fold', fold, '==========')

    train = candidates.iloc[train_idx]
    val = candidates.iloc[valid_idx]


    train = train.sort_values('session')
    groupsTrain = train.groupby('session').aid.agg('count').values
    dtrain = xgb.DMatrix(train.iloc[:, 2:-1], train.iloc[:, -1], group=groupsTrain)

    val = val.sort_values('session')
    groupsVal = val.groupby('session').aid.agg('count').values
    dvalid = xgb.DMatrix(val.iloc[:, 2:-1], val.iloc[:, -1], group=groupsVal)

    xgb_parms = {'objective':'rank:ndcg', 'tree_method':'gpu_hist'}
    model = xgb.train(xgb_parms, 
        dtrain=dtrain,
        evals=[(dtrain,'train'),(dvalid,'valid')],
        num_boost_round=1000,
        verbose_eval=100)
    model.save_model(modelSavedPAth + modelName + f'_fold{fold}.xgb')

    # Validate on 1 fold
    dvalid = xgb.DMatrix(val.iloc[:, 2:-1])
    pred = model.predict(dvalid)
    predsVal[valid_idx] = pred

========= type orders ==========
========= fold 0 ==========
[0]	train-map:0.97616	valid-map:0.97588
[100]	train-map:0.97861	valid-map:0.97733
[200]	train-map:0.97945	valid-map:0.97766
[300]	train-map:0.98009	valid-map:0.97771
[400]	train-map:0.98060	valid-map:0.97778
[500]	train-map:0.98100	valid-map:0.97776
[600]	train-map:0.98140	valid-map:0.97777
[700]	train-map:0.98176	valid-map:0.97778
[800]	train-map:0.98206	valid-map:0.97782
[900]	train-map:0.98235	valid-map:0.97780
[999]	train-map:0.98263	valid-map:0.97780
========= type orders ==========
========= fold 1 ==========
[0]	train-map:0.97603	valid-map:0.97596
[100]	train-map:0.97856	valid-map:0.97765
[200]	train-map:0.97942	valid-map:0.97785
[300]	train-map:0.98009	valid-map:0.97796
[400]	train-map:0.98058	valid-map:0.97800
[500]	train-map:0.98102	valid-map:0.97806
[600]	train-map:0.98139	valid-map:0.97805
[700]	train-map:0.98176	valid-map:0.97810
[800]	train-map:0.98207	valid-map:0.97813
[900]	train-map:0.98243	valid-map:0.97813


In [34]:
predictions = candidates[['session','aid']].copy()
predictions['pred'] = predsVal
predictions

,session,aid,pred
0,8643224,1502227,0.270591
1,8643224,945574,0.146390
2,8643224,631899,1.441910
3,8643226,1845885,1.317830
4,8643229,1696741,1.403060
...,...,...,...
5873476,9635951,1603001,-0.460546
5873477,9662935,767566,0.443818
5873478,9466602,574024,0.040874
5873479,9226021,361870,-0.493550


In [35]:
predictions = predictions.sort_values(['session','pred'], ascending=[True,False]).reset_index(drop=True)
predictions['n'] = predictions.groupby('session').aid.cumcount().astype('int8')
predictions = predictions.loc[predictions.n<20]
predictions

,session,aid,pred,n
0,8643220,1283505,1.700250,0
1,8643220,1355636,0.464761,1
2,8643220,199994,0.342867,2
3,8643222,422902,0.960622,0
4,8643222,485847,0.850568,1
...,...,...,...,...
5873476,11098483,965803,0.827493,2
5873477,11098483,511365,-1.284099,3
5873478,11098494,351417,0.488808,0
5873479,11098494,467632,-2.734641,1


In [36]:
sub = predictions.groupby('session').aid.apply(list)
sub = sub.to_frame().reset_index()
sub['type'] = predictType
sub.columns = ['session', 'labels', 'type']
sub

,session,labels,type
0,8643220,"[1283505, 1355636, 199994]",orders
1,8643222,"[422902, 485847, 662084, 288125]",orders
2,8643224,"[631899, 1256197, 1502227, 945574, 63847, 1824...",orders
3,8643225,"[1335839, 352192, 208525, 655488, 702179, 361270]",orders
4,8643226,"[1845885, 15252, 254273]",orders
...,...,...,...
2079599,11098472,"[1664927, 606307]",orders
2079600,11098478,[695879],orders
2079601,11098483,"[331673, 1267631, 965803, 511365]",orders
2079602,11098494,"[351417, 467632]",orders


In [21]:
subClicks = sub
subClicks

,session,labels,type
0,8643220,"[1532781, 1189975, 1602510]",clicks
1,8643221,[1535667],clicks
2,8643222,"[1414418, 13552, 559607, 208430]",clicks
3,8643223,"[1734305, 785712, 727928, 147526, 713187]",clicks
4,8643224,"[658590, 403869, 1289718]",clicks
...,...,...,...
2182192,11098472,[621352],clicks
2182193,11098478,"[564270, 1215173, 466846, 695879, 261405]",clicks
2182194,11098483,"[1825776, 369921, 499376, 1841350, 46115]",clicks
2182195,11098494,"[237349, 982280, 1185419, 1284622, 1562015, 17...",clicks


In [29]:
subCarts = sub
subCarts

,session,labels,type
0,8643220,"[1329320, 1190361, 935830]",carts
1,8643221,[1093709],carts
2,8643222,[176358],carts
3,8643223,"[1068136, 1116069]",carts
4,8643224,"[1365733, 10810, 799356, 403869]",carts
...,...,...,...
2086866,11098472,"[317654, 925086, 585972, 491766, 1008624]",carts
2086867,11098478,"[1522329, 1830718, 1371347]",carts
2086868,11098483,"[1124558, 309278]",carts
2086869,11098494,"[1733630, 366996, 256840]",carts


In [37]:
subOrders = sub
subOrders

,session,labels,type
0,8643220,"[1283505, 1355636, 199994]",orders
1,8643222,"[422902, 485847, 662084, 288125]",orders
2,8643224,"[631899, 1256197, 1502227, 945574, 63847, 1824...",orders
3,8643225,"[1335839, 352192, 208525, 655488, 702179, 361270]",orders
4,8643226,"[1845885, 15252, 254273]",orders
...,...,...,...
2079599,11098472,"[1664927, 606307]",orders
2079600,11098478,[695879],orders
2079601,11098483,"[331673, 1267631, 965803, 511365]",orders
2079602,11098494,"[351417, 467632]",orders


In [38]:
pred_df = pd.concat([subClicks, subCarts, subOrders])
pred_df

,session,labels,type
0,8643220,"[1532781, 1189975, 1602510]",clicks
1,8643221,[1535667],clicks
2,8643222,"[1414418, 13552, 559607, 208430]",clicks
3,8643223,"[1734305, 785712, 727928, 147526, 713187]",clicks
4,8643224,"[658590, 403869, 1289718]",clicks
...,...,...,...
2079599,11098472,"[1664927, 606307]",orders
2079600,11098478,[695879],orders
2079601,11098483,"[331673, 1267631, 965803, 511365]",orders
2079602,11098494,"[351417, 467632]",orders


In [39]:
test_labels = pd.read_parquet(inputPath + 'val_B.parquet')
test_labels

,session,type,ground_truth
0,8643220,clicks,[1189975]
1,8643221,clicks,[111691]
2,8643222,clicks,"[1104129, 1568399, 1772730, 1772730, 1104129]"
3,8643223,clicks,[1742315]
4,8643224,clicks,[471589]
...,...,...,...
2756163,11098472,clicks,[504876]
2756164,11098478,clicks,[261405]
2756165,11098483,clicks,[46115]
2756166,11098494,clicks,[1733630]


In [40]:
test_labels = test_labels.merge(pred_df, how='left', on=['session', 'type'])
test_labels

,session,type,ground_truth,labels
0,8643220,clicks,[1189975],"[1532781, 1189975, 1602510]"
1,8643221,clicks,[111691],[1535667]
2,8643222,clicks,"[1104129, 1568399, 1772730, 1772730, 1104129]","[1414418, 13552, 559607, 208430]"
3,8643223,clicks,[1742315],"[1734305, 785712, 727928, 147526, 713187]"
4,8643224,clicks,[471589],"[658590, 403869, 1289718]"
...,...,...,...,...
2756163,11098472,clicks,[504876],[621352]
2756164,11098478,clicks,[261405],"[564270, 1215173, 466846, 695879, 261405]"
2756165,11098483,clicks,[46115],"[1825776, 369921, 499376, 1841350, 46115]"
2756166,11098494,clicks,[1733630],"[237349, 982280, 1185419, 1284622, 1562015, 17..."


In [41]:
def intersect(df):
    if type(df.labels) == float:
        return 0
    else:
        return len(set(df.ground_truth).intersection(set(df.labels)))

In [42]:
test_labels['hits'] = test_labels.apply(lambda df: intersect(df), axis=1)
test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)
test_labels

,session,type,ground_truth,labels,hits,gt_count
0,8643220,clicks,[1189975],"[1532781, 1189975, 1602510]",1,1
1,8643221,clicks,[111691],[1535667],0,1
2,8643222,clicks,"[1104129, 1568399, 1772730, 1772730, 1104129]","[1414418, 13552, 559607, 208430]",0,5
3,8643223,clicks,[1742315],"[1734305, 785712, 727928, 147526, 713187]",0,1
4,8643224,clicks,[471589],"[658590, 403869, 1289718]",0,1
...,...,...,...,...,...,...
2756163,11098472,clicks,[504876],[621352],0,1
2756164,11098478,clicks,[261405],"[564270, 1215173, 466846, 695879, 261405]",1,1
2756165,11098483,clicks,[46115],"[1825776, 369921, 499376, 1841350, 46115]",1,1
2756166,11098494,clicks,[1733630],"[237349, 982280, 1185419, 1284622, 1562015, 17...",1,1


In [43]:
recall_per_type = test_labels.groupby(['type'])['hits'].sum() / test_labels.groupby(['type'])['gt_count'].sum()
recall_per_type

type
carts     0.412779
clicks    0.339695
orders    0.643386
dtype: float64

In [44]:
score = (recall_per_type * pd.Series({'clicks': 0.10, 'carts': 0.30, 'orders': 0.60})).sum()
score

0.5438349241010192

# Validation (Local testing)

## Create covisitation matrix

## Generate candidates for local testing data with covisitation

## Create features for local testing data

## Predict local testing data

In [5]:
data4xgb_path = '../output/data4xgb/'
predictType = 'orders'
data4xgb = pd.read_parquet(data4xgb_path + 'data4xgb_' + predictType + '.pqt')
data4xgb

,session,aid,item_item_count,item_user_count,item_buy_ratio,user_user_count,user_item_count,user_buy_ratio,order
0,11098528,11830,33776,19211,0.173970,1,1,0.0,1.0
1,11098528,1732105,10452,5770,0.280233,1,1,0.0,0.0
2,11098528,588923,24588,14741,0.125346,1,1,0.0,0.0
3,11098528,884502,31522,18163,0.135398,1,1,0.0,0.0
4,11098528,1157882,28341,16057,0.136975,1,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...
90062545,12899778,1097813,495,352,0.155556,1,1,0.0,0.0
90062546,12899778,1102546,1183,1001,0.087912,1,1,0.0,0.0
90062547,12899778,1103693,1514,950,0.058124,1,1,0.0,0.0
90062548,12899778,1249292,85,55,0.164706,1,1,0.0,0.0


In [6]:
modelSavedPAth = outputPath + 'savedModel/xgb/'
preds = np.zeros(len(data4xgb))
modelName = 'xgb_' + predictType
for fold in range(5):
    print('======= fold', fold, '===========')
    model = xgb.Booster()
    model.load_model(modelSavedPAth + modelName + f'_fold{fold}.xgb')
    model.set_param({'predictor': 'gpu_predictor'})
    dtest = xgb.DMatrix(data=data4xgb.iloc[:, 2:-1])
    preds += model.predict(dtest)/5
predictions = data4xgb[['session','aid']].copy()
predictions['pred'] = preds
predictions

======= fold 0 ===========
======= fold 1 ===========
======= fold 2 ===========
======= fold 3 ===========
======= fold 4 ===========


,session,aid,pred
0,11098528,11830,-0.527411
1,11098528,1732105,0.212281
2,11098528,588923,-0.396720
3,11098528,884502,-0.765019
4,11098528,1157882,-0.025167
...,...,...,...
90062545,12899778,1097813,0.337986
90062546,12899778,1102546,0.516349
90062547,12899778,1103693,0.005422
90062548,12899778,1249292,0.969177


In [7]:
predictions = predictions.sort_values(['session','pred'], ascending=[True,False]).reset_index(drop=True)
predictions['n'] = predictions.groupby('session').aid.cumcount().astype('int8')
predictions = predictions.loc[predictions.n<20]
predictions

,session,aid,pred,n
0,11098528,1633746,0.393000,0
1,11098528,1519088,0.368430,1
2,11098528,855613,0.295283,2
3,11098528,1517680,0.215068,3
4,11098528,1317291,0.212443,4
...,...,...,...,...
90062515,12899778,189165,0.584108,15
90062516,12899778,1102546,0.516349,16
90062517,12899778,1320960,0.504600,17
90062518,12899778,694776,0.491851,18


In [23]:
sub = predictions.groupby('session').aid.apply(list)
sub = sub.to_frame().reset_index()
sub['type'] = predictType
sub.columns = ['session', 'labels', 'type']
sub

,session,labels,type
0,11098528,"[1633746, 1519088, 855613, 1517680, 1317291, 1...",orders
1,11098529,"[400271, 41093, 294355, 890962, 1140565, 18016...",orders
2,11098530,"[82493, 335674, 29445, 975481, 460553, 745689,...",orders
3,11098531,"[698990, 1032999, 1449555, 1309633, 1557766, 1...",orders
4,11098532,"[476681, 1720259, 669555, 1359971, 535538, 143...",orders
...,...,...,...
1801246,12899774,"[255895, 1584669, 669903, 1089446, 778604, 164...",orders
1801247,12899775,"[1604133, 1263710, 1469978, 823143, 310546, 63...",orders
1801248,12899776,"[861477, 1650734, 334325, 360421, 748226, 1330...",orders
1801249,12899777,"[1368794, 1497245, 1751703, 1250945, 1486067, ...",orders


In [12]:
subClicks = sub
subClicks

,session,labels,type
0,11098528,"[1633746, 1519088, 1517680, 1586171, 571762, 6...",clicks
1,11098529,"[924195, 1643015, 121149, 333991, 528290, 1384...",clicks
2,11098530,"[335674, 82493, 1604220, 841206, 29445, 460553...",clicks
3,11098531,"[698990, 1449555, 605877, 1075480, 1557766, 13...",clicks
4,11098532,"[108125, 994449, 476681, 879146, 876469, 75819...",clicks
...,...,...,...
1801246,12899774,"[743977, 1053112, 861433, 1536539, 454910, 127...",clicks
1801247,12899775,"[1263710, 1604133, 329725, 1255910, 1760714, 1...",clicks
1801248,12899776,"[1590416, 861477, 1079559, 793373, 109949, 773...",clicks
1801249,12899777,"[1368794, 1486067, 144136, 1515273, 1783021, 5...",clicks


In [18]:
subCarts = sub
subCarts

,session,labels,type
0,11098528,"[1633746, 1519088, 322370, 1853703, 1732105, 6...",carts
1,11098529,"[400271, 727009, 1049061, 294355, 41093, 18016...",carts
2,11098530,"[82493, 335674, 1596897, 29445, 460553, 150661...",carts
3,11098531,"[393436, 653899, 1449555, 1309633, 1032999, 33...",carts
4,11098532,"[476681, 1720259, 496180, 876469, 1359971, 143...",carts
...,...,...,...
1801246,12899774,"[1584669, 255895, 778604, 1089446, 1648828, 66...",carts
1801247,12899775,"[1604133, 1263710, 1436280, 823143, 155954, 49...",carts
1801248,12899776,"[861477, 1100749, 748226, 643477, 1650734, 334...",carts
1801249,12899777,"[1497245, 1486067, 1250945, 1196256, 1783021, ...",carts


In [24]:
subOrders = sub
subOrders

,session,labels,type
0,11098528,"[1633746, 1519088, 855613, 1517680, 1317291, 1...",orders
1,11098529,"[400271, 41093, 294355, 890962, 1140565, 18016...",orders
2,11098530,"[82493, 335674, 29445, 975481, 460553, 745689,...",orders
3,11098531,"[698990, 1032999, 1449555, 1309633, 1557766, 1...",orders
4,11098532,"[476681, 1720259, 669555, 1359971, 535538, 143...",orders
...,...,...,...
1801246,12899774,"[255895, 1584669, 669903, 1089446, 778604, 164...",orders
1801247,12899775,"[1604133, 1263710, 1469978, 823143, 310546, 63...",orders
1801248,12899776,"[861477, 1650734, 334325, 360421, 748226, 1330...",orders
1801249,12899777,"[1368794, 1497245, 1751703, 1250945, 1486067, ...",orders


In [25]:
pred_df = pd.concat([subClicks, subCarts, subOrders])
pred_df

,session,labels,type
0,11098528,"[1633746, 1519088, 1517680, 1586171, 571762, 6...",clicks
1,11098529,"[924195, 1643015, 121149, 333991, 528290, 1384...",clicks
2,11098530,"[335674, 82493, 1604220, 841206, 29445, 460553...",clicks
3,11098531,"[698990, 1449555, 605877, 1075480, 1557766, 13...",clicks
4,11098532,"[108125, 994449, 476681, 879146, 876469, 75819...",clicks
...,...,...,...
1801246,12899774,"[255895, 1584669, 669903, 1089446, 778604, 164...",orders
1801247,12899775,"[1604133, 1263710, 1469978, 823143, 310546, 63...",orders
1801248,12899776,"[861477, 1650734, 334325, 360421, 748226, 1330...",orders
1801249,12899777,"[1368794, 1497245, 1751703, 1250945, 1486067, ...",orders


In [6]:
candidatesSavePath = '../output/top50Candidates/'
clicks_pred_df = pd.read_parquet(candidatesSavePath + 'clicksCandidate.pqt')
buys_pred_df = pd.read_parquet(candidatesSavePath + 'buysCandidate.pqt')

clicks_pred_df['session'] = clicks_pred_df.session.astype('int32')
clicks_pred_df['labels'] = clicks_pred_df.labels.apply(lambda x: x.astype(np.int32))
buys_pred_df['session'] = buys_pred_df.session.astype('int32')
buys_pred_df['labels'] = buys_pred_df.labels.apply(lambda x: x.astype(np.int32))

clicks_pred_df

,session,labels
0,11098528,"[11830, 588923, 1732105, 571762, 884502, 11578..."
1,11098529,"[1105029, 459126, 1339838, 1544564, 217742, 16..."
2,11098530,"[409236, 264500, 1603001, 963957, 254154, 5830..."
3,11098531,"[396199, 1271998, 452188, 1728212, 1365569, 62..."
4,11098532,"[876469, 7651, 108125, 612920, 1673641, 120261..."
...,...,...
1801246,12899774,"[33035, 1539309, 819288, 95488, 771913, 270852..."
1801247,12899775,"[1743151, 1760714, 1255910, 1163166, 832192, 2..."
1801248,12899776,"[548599, 1401030, 1440959, 1607333, 1144446, 3..."
1801249,12899777,"[384045, 1308634, 1688215, 703474, 395762, 148..."


In [7]:
clicks_pred_df['type'] = 'clicks'

,session,labels,type
0,11098528,"[11830, 588923, 1732105, 571762, 884502, 11578...",clicks
1,11098529,"[1105029, 459126, 1339838, 1544564, 217742, 16...",clicks
2,11098530,"[409236, 264500, 1603001, 963957, 254154, 5830...",clicks
3,11098531,"[396199, 1271998, 452188, 1728212, 1365569, 62...",clicks
4,11098532,"[876469, 7651, 108125, 612920, 1673641, 120261...",clicks
...,...,...,...
1801246,12899774,"[33035, 1539309, 819288, 95488, 771913, 270852...",clicks
1801247,12899775,"[1743151, 1760714, 1255910, 1163166, 832192, 2...",clicks
1801248,12899776,"[548599, 1401030, 1440959, 1607333, 1144446, 3...",clicks
1801249,12899777,"[384045, 1308634, 1688215, 703474, 395762, 148...",clicks


In [8]:
carts_pred_df = buys_pred_df.copy()
carts_pred_df['type'] = 'carts'
carts_pred_df

,session,labels,type
0,11098528,"[11830, 1732105, 588923, 884502, 1157882, 5717...",carts
1,11098529,"[1105029, 1049489, 132016, 459126, 785427, 792...",carts
2,11098530,"[409236, 264500, 1603001, 254154, 963957, 5830...",carts
3,11098531,"[396199, 1271998, 452188, 1728212, 1365569, 62...",carts
4,11098532,"[876469, 7651, 108125, 612920, 1159379, 120261...",carts
...,...,...,...
1801246,12899774,"[33035, 1539309, 819288, 95488, 270852, 771913...",carts
1801247,12899775,"[1743151, 1760714, 1163166, 1255910, 1498443, ...",carts
1801248,12899776,"[548599, 1144446, 1440959, 1607333, 695829, 14...",carts
1801249,12899777,"[384045, 1308634, 1688215, 395762, 1486067, 70...",carts


In [9]:
buys_pred_df['type'] = 'orders'
buys_pred_df

,session,labels,type
0,11098528,"[11830, 1732105, 588923, 884502, 1157882, 5717...",orders
1,11098529,"[1105029, 1049489, 132016, 459126, 785427, 792...",orders
2,11098530,"[409236, 264500, 1603001, 254154, 963957, 5830...",orders
3,11098531,"[396199, 1271998, 452188, 1728212, 1365569, 62...",orders
4,11098532,"[876469, 7651, 108125, 612920, 1159379, 120261...",orders
...,...,...,...
1801246,12899774,"[33035, 1539309, 819288, 95488, 270852, 771913...",orders
1801247,12899775,"[1743151, 1760714, 1163166, 1255910, 1498443, ...",orders
1801248,12899776,"[548599, 1144446, 1440959, 1607333, 695829, 14...",orders
1801249,12899777,"[384045, 1308634, 1688215, 395762, 1486067, 70...",orders


In [10]:
pred_df = pd.concat([clicks_pred_df, carts_pred_df, buys_pred_df])
pred_df

,session,labels,type
0,11098528,"[11830, 588923, 1732105, 571762, 884502, 11578...",clicks
1,11098529,"[1105029, 459126, 1339838, 1544564, 217742, 16...",clicks
2,11098530,"[409236, 264500, 1603001, 963957, 254154, 5830...",clicks
3,11098531,"[396199, 1271998, 452188, 1728212, 1365569, 62...",clicks
4,11098532,"[876469, 7651, 108125, 612920, 1673641, 120261...",clicks
...,...,...,...
1801246,12899774,"[33035, 1539309, 819288, 95488, 270852, 771913...",orders
1801247,12899775,"[1743151, 1760714, 1163166, 1255910, 1498443, ...",orders
1801248,12899776,"[548599, 1144446, 1440959, 1607333, 695829, 14...",orders
1801249,12899777,"[384045, 1308634, 1688215, 395762, 1486067, 70...",orders


## Evaluate

In [11]:
test_labels = pd.read_parquet('../input/splited/test_labels.parquet')
test_labels

,session,type,ground_truth
0,11098528,clicks,[1679529]
1,11098528,carts,[1199737]
2,11098528,orders,"[990658, 950341, 1462506, 1561739, 907564, 369..."
3,11098529,clicks,[1105029]
4,11098530,orders,[409236]
...,...,...,...
2212687,12899774,clicks,[1399483]
2212688,12899775,clicks,[1760714]
2212689,12899776,clicks,[1737908]
2212690,12899777,clicks,[384045]


In [12]:
test_labels = test_labels.merge(pred_df, how='left', on=['session', 'type'])
test_labels

,session,type,ground_truth,labels
0,11098528,clicks,[1679529],"[11830, 588923, 1732105, 571762, 884502, 11578..."
1,11098528,carts,[1199737],"[11830, 1732105, 588923, 884502, 1157882, 5717..."
2,11098528,orders,"[990658, 950341, 1462506, 1561739, 907564, 369...","[11830, 1732105, 588923, 884502, 1157882, 5717..."
3,11098529,clicks,[1105029],"[1105029, 459126, 1339838, 1544564, 217742, 16..."
4,11098530,orders,[409236],"[409236, 264500, 1603001, 254154, 963957, 5830..."
...,...,...,...,...
2212687,12899774,clicks,[1399483],"[33035, 1539309, 819288, 95488, 771913, 270852..."
2212688,12899775,clicks,[1760714],"[1743151, 1760714, 1255910, 1163166, 832192, 2..."
2212689,12899776,clicks,[1737908],"[548599, 1401030, 1440959, 1607333, 1144446, 3..."
2212690,12899777,clicks,[384045],"[384045, 1308634, 1688215, 703474, 395762, 148..."


In [19]:
test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)
test_labels

,session,type,ground_truth,labels,hits,gt_count
0,11098528,clicks,[1679529],"[11830, 588923, 1732105, 571762, 884502, 11578...",0,1
1,11098528,carts,[1199737],"[11830, 1732105, 588923, 884502, 1157882, 5717...",0,1
2,11098528,orders,"[990658, 950341, 1462506, 1561739, 907564, 369...","[11830, 1732105, 588923, 884502, 1157882, 5717...",1,11
3,11098529,clicks,[1105029],"[1105029, 459126, 1339838, 1544564, 217742, 16...",1,1
4,11098530,orders,[409236],"[409236, 264500, 1603001, 254154, 963957, 5830...",1,1
...,...,...,...,...,...,...
2212687,12899774,clicks,[1399483],"[33035, 1539309, 819288, 95488, 771913, 270852...",1,1
2212688,12899775,clicks,[1760714],"[1743151, 1760714, 1255910, 1163166, 832192, 2...",1,1
2212689,12899776,clicks,[1737908],"[548599, 1401030, 1440959, 1607333, 1144446, 3...",1,1
2212690,12899777,clicks,[384045],"[384045, 1308634, 1688215, 703474, 395762, 148...",1,1


In [20]:
recall_per_type = test_labels.groupby(['type'])['hits'].sum() / test_labels.groupby(['type'])['gt_count'].sum()
recall_per_type

type
carts     0.463194
clicks    0.601151
orders    0.682078
dtype: float64

In [21]:
score = (recall_per_type * pd.Series({'clicks': 0.10, 'carts': 0.30, 'orders': 0.60})).sum()
score

0.60831993689032

# Predict (Online testing)

In [14]:
data4xgb_path = '../output/data4xgbAllTrain/'
predictType = 'orders'
if predictType == 'clicks':
    data4xgb = pd.read_parquet(data4xgb_path + 'data4xgb.pqt')
else:
    data4xgb = pd.read_parquet(data4xgb_path + 'data4xgb_buys.pqt')
data4xgb

,session,aid,item_item_count,item_user_count,item_buy_ratio,user_user_count,user_item_count,user_buy_ratio
0,12899779,59625,13,12,0.000000,1,1,0.0
1,12899779,397451,138,69,0.239130,1,1,0.0
2,12899779,469285,45,27,0.066667,1,1,0.0
3,12899779,1493965,185,134,0.059459,1,1,0.0
4,12899779,438191,3651,2305,0.033416,1,1,0.0
...,...,...,...,...,...,...,...,...
83590145,14571581,234109,2423,1544,0.043335,1,1,0.0
83590146,14571581,724491,4696,3067,0.050256,1,1,0.0
83590147,14571581,747616,5468,3219,0.046086,1,1,0.0
83590148,14571581,870028,1350,914,0.043704,1,1,0.0


In [15]:
modelSavedPAth = outputPath + 'savedModel/xgb/'
preds = np.zeros(len(data4xgb))
modelName = 'xgb_' + predictType
for fold in range(5):
    print('======= fold', fold, '===========')
    model = xgb.Booster()
    model.load_model(modelSavedPAth + modelName + f'_fold{fold}.xgb')
    model.set_param({'predictor': 'gpu_predictor'})
    dtest = xgb.DMatrix(data=data4xgb.iloc[:, 2:-1])
    preds += model.predict(dtest)/5
predictions = data4xgb[['session','aid']].copy()
predictions['pred'] = preds
predictions

======= fold 0 ===========
======= fold 1 ===========
======= fold 2 ===========
======= fold 3 ===========
======= fold 4 ===========


,session,aid,pred
0,12899779,59625,-1.912859
1,12899779,397451,1.273146
2,12899779,469285,0.144814
3,12899779,1493965,-0.345668
4,12899779,438191,-1.223110
...,...,...,...
83590145,14571581,234109,-0.904425
83590146,14571581,724491,-0.457194
83590147,14571581,747616,-0.524583
83590148,14571581,870028,-1.051970


In [16]:
predictions = predictions.sort_values(['session','pred'], ascending=[True,False]).reset_index(drop=True)
predictions['n'] = predictions.groupby('session').aid.cumcount().astype('int8')
predictions = predictions.loc[predictions.n<20]

sub = predictions.groupby('session').aid.apply(list)
sub = sub.to_frame().reset_index()
sub.aid = sub.aid.apply(lambda x: " ".join(map(str,x)))
sub.columns = ['session_type','labels']
sub.session_type = sub.session_type.astype('str')+ '_' + predictType
sub

,session_type,labels
0,12899779_orders,679602 696438 448688 731692 397451 686765 1790...
1,12899780_orders,760500 973453 1290032 1162514 1517468 1493044 ...
2,12899781_orders,129797 1681537 1200570 950718 811084 918667 19...
3,12899782_orders,925759 834354 638410 779477 127404 1711180 303...
4,12899783_orders,1817895 1492009 1729553 255297 354698 379769 9...
...,...,...
1671798,14571577_orders,30148 16184 1432586 1768884 1273700 1088524 63...
1671799,14571578_orders,1403962 178547 1572352 131032 231894 856256 52...
1671800,14571579_orders,785544 164523 702275 398437 137494 560081 9070...
1671801,14571580_orders,1357971 887920 202353 1192366 1246483 178628 1...


In [9]:
subClicks = sub
subClicks

,session_type,labels
0,12899779_clicks,679602 731692 696438 475447 620510 1700255 179...
1,12899780_clicks,760500 103974 231487 636813 1796048 1586171 16...
2,12899781_clicks,447645 1681537 1495531 374037 734026 1836671 8...
3,12899782_clicks,229748 603159 987399 595994 1699932 303142 779...
4,12899783_clicks,1817895 198385 1492009 1157882 1729553 1062554...
...,...,...
1671798,14571577_clicks,459882 447748 1273700 1768884 1088524 1190776 ...
1671799,14571578_clicks,231894 1109341 1512002 455319 1400242 1631939 ...
1671800,14571579_clicks,426321 1189853 177070 1550479 1498799 1607245 ...
1671801,14571580_clicks,1652005 1455166 202353 462655 32322 682237 560...


In [13]:
subCarts = sub
subCarts

,session_type,labels
0,12899779_carts,679602 696438 397451 620510 1027207 159257 731...
1,12899780_carts,209046 1493044 404612 1517468 1717153 636813 6...
2,12899781_carts,141736 199008 1200570 811084 747697 1006790 11...
3,12899782_carts,1445562 1696036 794259 1711180 1352725 834354 ...
4,12899783_carts,198385 1817895 1060039 1729553 230028 354698 9...
...,...,...
1671798,14571577_carts,63870 21264 1432586 1273700 16184 86916 152316...
1671799,14571578_carts,131032 178547 1572352 400149 934566 231894 326...
1671800,14571579_carts,739876 374843 569414 398437 164523 988492 3047...
1671801,14571580_carts,391852 1324905 1675403 682237 178628 473806 54...


In [17]:
subOrders = sub
subOrders

,session_type,labels
0,12899779_orders,679602 696438 448688 731692 397451 686765 1790...
1,12899780_orders,760500 973453 1290032 1162514 1517468 1493044 ...
2,12899781_orders,129797 1681537 1200570 950718 811084 918667 19...
3,12899782_orders,925759 834354 638410 779477 127404 1711180 303...
4,12899783_orders,1817895 1492009 1729553 255297 354698 379769 9...
...,...,...
1671798,14571577_orders,30148 16184 1432586 1768884 1273700 1088524 63...
1671799,14571578_orders,1403962 178547 1572352 131032 231894 856256 52...
1671800,14571579_orders,785544 164523 702275 398437 137494 560081 9070...
1671801,14571580_orders,1357971 887920 202353 1192366 1246483 178628 1...


In [18]:
pred_df = pd.concat([subClicks, subCarts, subOrders])
pred_df

,session_type,labels
0,12899779_clicks,679602 731692 696438 475447 620510 1700255 179...
1,12899780_clicks,760500 103974 231487 636813 1796048 1586171 16...
2,12899781_clicks,447645 1681537 1495531 374037 734026 1836671 8...
3,12899782_clicks,229748 603159 987399 595994 1699932 303142 779...
4,12899783_clicks,1817895 198385 1492009 1157882 1729553 1062554...
...,...,...
1671798,14571577_orders,30148 16184 1432586 1768884 1273700 1088524 63...
1671799,14571578_orders,1403962 178547 1572352 131032 231894 856256 52...
1671800,14571579_orders,785544 164523 702275 398437 137494 560081 9070...
1671801,14571580_orders,1357971 887920 202353 1192366 1246483 178628 1...


In [19]:
pred_df.to_csv("../submissions/rerankWithXgb_withVal.csv", index=False)